In [1]:
!pip install datasets transformers peft accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
#huggingface login
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `absa3` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate wh

In [9]:
import torch
import ast
from datasets import load_dataset
from transformers import LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig

In [10]:
def load_data():
    dataset = load_dataset("csv", data_files={
        "train": "/content/drive/MyDrive/T5_finetuning/Dataset2/Laptop2014/Train/Laptops2014_Train_Formatted.csv",
        "validation": "/content/drive/MyDrive/T5_finetuning/Dataset2/Laptop2014/Trial/laptops_Trial_Formatted.csv",
        "test": "/content/drive/MyDrive/T5_finetuning/Dataset2/Laptop2014/Test/Laptops_Test_Gold_Formatted.csv"
    })
    return dataset


In [11]:
def predict_aspects(model, tokenizer, text):
    """
    Uses the same prompt as in finetuning to get the model's output.
    """
    prompt = (
        "Below is a product/service-related text. "
        "Identify any aspects and their sentiment polarity.\n"
        "Return the result as 'aspect:polarity' pairs. If none exist, return 'noaspectterms'.\n\n"
        f"Text:\n{text}\n\nAnswer:\n"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        generation = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )
    output = tokenizer.decode(generation[0], skip_special_tokens=True)

    # Extract content after 'Answer:'
    if "Answer:" in output:
        output = output.split("Answer:", 1)[1]
    # Trim leading spaces on the first line
    lines = output.split("\n")
    if lines:
        lines[0] = lines[0].lstrip()
    output = "\n".join(lines)
    return output

In [12]:
def parse_gold_aspect_pairs(aspect_str):
    if aspect_str == "noaspectterms":
        return set()
    try:
        data = ast.literal_eval(aspect_str)
    except:
        return set()

    aspect_set = set()
    for d in data:
        term = d.get('term', '').strip().lower()
        polarity = d.get('polarity', '').strip().lower()
        if term and polarity:
            aspect_set.add((term, polarity))
    return aspect_set

def parse_pred_aspect_pairs(pred_str):
    lines = pred_str.strip().splitlines()
    if len(lines) == 1 and lines[0].strip().lower() == "noaspectterms":
        return set()

    aspect_set = set()
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if ':' in line:
            aspect, polarity = line.split(':', 1)
            aspect_set.add((aspect.strip().lower(), polarity.strip().lower()))
    return aspect_set

def compute_exact_match_accuracy(y_true_sets, y_pred_sets):
    correct = 0
    for gold_set, pred_set in zip(y_true_sets, y_pred_sets):
        if gold_set == pred_set:
            correct += 1
    return correct / len(y_true_sets)

def compute_micro_stats(y_true_sets, y_pred_sets):
    tp = 0
    total_gold = 0
    total_pred = 0
    for gold_set, pred_set in zip(y_true_sets, y_pred_sets):
        tp += len(gold_set.intersection(pred_set))
        total_gold += len(gold_set)
        total_pred += len(pred_set)

    precision = tp / total_pred if total_pred > 0 else 0.0
    recall = tp / total_gold if total_gold > 0 else 0.0
    if precision + recall == 0:
        f1 = 0.0
    else:
        f1 = 2 * precision * recall / (precision + recall)
    return precision, recall, f1

def compute_metrics(y_true_sets, y_pred_sets):
    accuracy = compute_exact_match_accuracy(y_true_sets, y_pred_sets)
    precision, recall, f1 = compute_micro_stats(y_true_sets, y_pred_sets)
    return {
        "accuracy_exact_match": accuracy,
        "precision_micro": precision,
        "recall_micro": recall,
        "f1_micro": f1
    }

def evaluate_on_test(test_dataset, model, tokenizer):
    y_true_sets = []
    y_pred_sets = []
    for ex in test_dataset:
        gold_set = parse_gold_aspect_pairs(ex['aspectTerms'])
        y_true_sets.append(gold_set)

        pred_str = predict_aspects(model, tokenizer, ex['raw_text'])
        pred_set = parse_pred_aspect_pairs(pred_str)
        y_pred_sets.append(pred_set)

    results = compute_metrics(y_true_sets, y_pred_sets)
    return results

In [15]:
def main():
    # Load the dataset
    dataset = load_data()
    test_data = dataset["test"]

    # Load the base Llama 2 7B model in 4-bit
    base_model_id = "meta-llama/Llama-2-7b-hf"
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )
    model = LlamaForCausalLM.from_pretrained(
        base_model_id,
        quantization_config=bnb_config,
        device_map="auto"
    )
    tokenizer = LlamaTokenizer.from_pretrained(base_model_id)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Evaluate the raw, un-finetuned model on the test set
    results = evaluate_on_test(test_data, model, tokenizer)
    print("Evaluation results on base Llama 2 7B (no fine-tuning):")
    print(results)

if __name__ == "__main__":
    main()

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Evaluation results on base Llama 2 7B (no fine-tuning):
{'accuracy_exact_match': 0.035, 'precision_micro': 0.006185567010309278, 'recall_micro': 0.018376722817764167, 'f1_micro': 0.009255688391824142}




*   'accuracy_exact_match': 0.035,
*    'precision_micro': 0.006185567010309278,
*  'recall_micro': 0.018376722817764167,
* 'f1_micro': 0.009255688391824142




In [17]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig
from peft import PeftModel  # not strictly needed here, but you might already have it installed

# The same prompt-based function as before
def predict_aspects(model, tokenizer, text):
    prompt = (
        "Below is a product/service-related text. "
        "Identify any aspects and their sentiment polarity.\n"
        "Return the result as 'aspect:polarity' pairs. If none exist, return 'noaspectterms'.\n\n"
        f"Text:\n{text}\n\nAnswer:\n"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        generation = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )
    output = tokenizer.decode(generation[0], skip_special_tokens=True)

    # Extract text after "Answer:"
    if "Answer:" in output:
        output = output.split("Answer:", 1)[1]
    # Remove leading spaces from the first line
    lines = output.split("\n")
    if lines:
        lines[0] = lines[0].lstrip()
    output = "\n".join(lines)
    return output

#############################################
# 1. Load Base Llama 2 7B
#############################################

base_model_id = "meta-llama/Llama-2-7b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = LlamaForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
tokenizer = LlamaTokenizer.from_pretrained(base_model_id)
# Ensure we have a pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

#############################################
# 2. Single Inference
#############################################

test_text = "The screen quality is excellent, but the battery life is disappointing."
prediction = predict_aspects(model, tokenizer, test_text)

print("Input text: ", test_text)
print("Model Output:\n", prediction)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Input text:  The screen quality is excellent, but the battery life is disappointing.
Model Output:
 
noaspectterms

Sentiment:
Neutral

Sentiment Polarity:
Positive

```python
from nltk.sentiment.vader import SentimentAnalyzer


